# Advent of Code 2015 - Day 22

This was one of the more interesting problems. I did have to consult Reddit for some hints. I finally wrote a brute-force recursive solution. 

I kept a list of winning costs, and skipped games that accrued more mana than a won game that cost less. To manage complexity, I capture the game state in an object (a la JavaScript rather than defining a class) and pass it to functions that modify it.

This could probably be made much more efficient by casting the problem as a shortest path problem and solving by Dijkstra's algorithm.

In [1]:
Spells = {
  'Magic Missile' : 53,
  'Drain' : 73,
  'Shield' : 113,
  'Poison' : 173,
  'Recharge' :  229,
}

Effects = ['Shield', 'Poison', 'Recharge']

## Part 1

In [2]:
import copy

def Player_Tick(GS, Spell, Hard = False):

  Game_State = copy.deepcopy(GS)

  if Hard:
    Game_State['Player']['Hit Points'] -= 1
    if Game_State['Boss']['Hit Points'] <= 0:
      Game_State['Status'] = 'Player Won'
      return Game_State
    if Game_State['Player']['Hit Points'] <= 0:
      Game_State['Status'] = 'Player Lost'
      return Game_State

  # If given spell is an "effective" active spell
  if Spell in Effects:
    if Game_State['Effects'][Spell] > 1:
      Game_State['Status'] = 'Player Lost' # Technically, the player does not lose, the move is forbidden
      return Game_State

  # If we cannot afford given spell
  if Spells[Spell] > Game_State['Player']['Mana']:
    Game_State['Status'] = 'Player Lost'
    return Game_State
  
  # Apply effects
  for effect in Game_State['Effects']:
    if Game_State['Effects'][effect]:
      Game_State['Effects'][effect] -= 1

      if effect == 'Shield':
        if Game_State['Effects'][effect] == 0:
          Game_State['Player']['Armor'] -= 7
      
      elif effect == 'Poison':
        Game_State['Boss']['Hit Points'] -= 3

      elif effect == 'Recharge':
        Game_State['Player']['Mana'] += 101


  # Check won loss due to effects
  if Game_State['Boss']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Won'
    return Game_State
  if Game_State['Player']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Lost'
    return Game_State

  # Apply Current Spell
  if Spell == 'Magic Missile':
    Game_State['Player']['Mana'] -= Spells[Spell]
    Game_State['Boss']['Hit Points'] -= 4

  elif Spell == 'Drain':
    Game_State['Player']['Mana'] -= Spells[Spell]
    Game_State['Boss']['Hit Points'] -= 2
    Game_State['Player']['Hit Points'] += 2

  elif Spell == 'Shield':
    Game_State['Player']['Mana'] -= Spells[Spell]
    Game_State['Player']['Armor'] += 7
    Game_State['Effects']['Shield'] = 6

  elif Spell == 'Poison':
    Game_State['Player']['Mana'] -= Spells[Spell]
    Game_State['Effects']['Poison'] = 6

  elif Spell == 'Recharge':
    Game_State['Player']['Mana'] -= Spells[Spell]
    Game_State['Effects']['Recharge'] = 5

  # Check won loss due to new spells
  if Game_State['Boss']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Won'
  if Game_State['Player']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Lost'

  return Game_State

In [3]:
def Boss_Tick(GS):

  Game_State = copy.deepcopy(GS)

  # Apply effects
  for effect in Game_State['Effects']:
    if Game_State['Effects'][effect]:
      Game_State['Effects'][effect] -= 1

      if effect == 'Shield':
        if Game_State['Effects'][effect] == 0:
          Game_State['Player']['Armor'] -= 7
      
      elif effect == 'Poison':
        Game_State['Boss']['Hit Points'] -= 3

      elif effect == 'Recharge':
        Game_State['Player']['Mana'] += 101

  # Check won loss due to effects
  if Game_State['Boss']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Won'
    return Game_State
  if Game_State['Player']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Lost'
    return Game_State

  deal = max(1, Game_State['Boss']['Damage'] - Game_State['Player']['Armor'])
  Game_State['Player']['Hit Points'] -= deal

  # Check won loss due to boss attack
  if Game_State['Boss']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Won'
  if Game_State['Player']['Hit Points'] <= 0:
    Game_State['Status'] = 'Player Lost'

  return Game_State

In [4]:
# Given a Game State and Spell
# Player plays a turn
# If game is not lost/won
# Boss plays a turn
# Return new game state

def Tick(GS, Spell, Hardness):
  Game_State = Player_Tick(GS, Spell, Hardness)
  if Game_State['Status'] == 'On Going':
    Game_State = Boss_Tick(Game_State)

  return Game_State

In [5]:
import math

def find_least_mana_to_spend_and_win(GS, depth, money_spent_so_far, hardness, successful):

  results = []
  for spell in Spells:
    money_spent = Spells[spell]
    new_GS = Tick(GS, spell, hardness)

    if new_GS['Status'] == 'Player Won':
      results.append(money_spent)
      successful.add(money_spent_so_far + money_spent)

    elif new_GS['Status'] == 'On Going':
      # Only try if a cheap winning series of moves was not found already
      if money_spent_so_far + money_spent < min(successful): 
        x = find_least_mana_to_spend_and_win(new_GS, depth + 1, money_spent_so_far + money_spent, hardness, successful)
        if x:
          results.append(money_spent + x)

  if results: 
    return min(results)

In [6]:
Game_State = {'Player' : {'Hit Points' : 10, 'Armor' : 0, 'Mana' : 250},
              'Boss' : {'Hit Points' : 13, 'Damage' : 8, 'Armor' : 0},
              'Effects' : {'Shield' : 0, 'Poison': 0, 'Recharge' : 0},
              'Status' : 'On Going'}

find_least_mana_to_spend_and_win(Game_State, 0, 0, False, set([math.inf]))

226

In [7]:
Game_State = {'Player' : {'Hit Points' : 50, 'Armor' : 0, 'Mana' : 500},
              'Boss' : {'Hit Points' : 55, 'Damage' : 8, 'Armor' : 0},
              'Effects' : {'Shield' : 0, 'Poison': 0, 'Recharge' : 0},
              'Status' : 'On Going'}
              
find_least_mana_to_spend_and_win(Game_State, 0, 0, False, set([math.inf]))

953

## Part 2

In [8]:
find_least_mana_to_spend_and_win(Game_State, 0, 0, True, set([math.inf]))

1289